<img src='../../media/common/LogoWekeo_Copernicus_RGB_0.png' align='left' height='96px'></img>

<hr>

# Tutorial on Basic Land Applications (Data Download)

In this tutorial, we will use the WEkEO Jupyterhub to access and analyze data from the Copernicus Sentinel-2 and the <a href='https://land.copernicus.eu/' target='_blank'>Copernicus Land Monitoring Service (CLMS)</a>.  
We have chosen a region in northern Corsica because it features representative landscape characteristics and processes that highlight the strengths and capabilities of Copernicus space components and services.

The tutorial includes the following steps:

1. [Search and download data](#search_download): We will select and download a Sentinel-2 scene and the CLMS CORINE Land Cover (CLC) data from their original archive locations via WEkEO using the Harmonised Data Access (HAD) API.

1.	Read and View Sentinel-2 Data: Once downloaded, we will read and view the Sentinel-2 data in geographic coordinates as a true-color image.

1.	Process and View Sentinel-2 Data as a Vegetation Index: We will learn how to assess vegetation density and health using optical Earth Observation data, which supports crop and landscape management practices.

1.	Read and View the CLC Data: Display the thematic CLC data with the correct legend.

1.	Analyze CLC2018 Burnt Area in the Sentinel-2 NDVI Data: We will combine the two products to provide a more comprehensive analysis, despite their differences.

<img src='../../media/land/Intro_banner.jpg' align='center' height='400px'></img>

## Environment Setup
Before we begin, we need to prepare our environment by installing and importing the necessary Python libraries.

### Install the WEkEO HDA Client

The WEkEO HDA client is a Python-based library that supports both Python 2.7.x and Python 3.

To install the WEkEO HDA client using the package management system pip, run the following command on Unix/Linux:

In [ ]:
pip install hda -U

### Load Required Libraries

Please verify that the following requirements are installed before proceeding to the next step:
   - `requests`
   - `tqdm`

In [59]:
import os
import sys
import json
import time
import base64
from IPython.core.display import HTML

import requests
import warnings
warnings.filterwarnings('ignore')
import zipfile

### Load WEkEO HDA client

The hda client provides a fully compliant Python 3 client that can be used to search and download products using the Harmonized Data Access WEkEO API.
HDA is RESTful interface allowing users to search and download WEkEO datasets.
Documentation about its usage can be found at the <a href='https://www.wekeo.eu/' target='_blank'>WEkEO website</a>.

In [60]:
from hda import Client

<hr>

## HDA Client Setup

### WEkEO Account Registration

If you don't have a WEkEO account, please self-register at the <a href='https://my.wekeo.eu/web/guest/user-registration' target='_blank'>WEkEO registration page</a>.

### API Authentication

To interact with WEkEO's Harmonised Data Access API, ensure the file `$HOME/.hdarc` exists with the URL to the API endpoint, and your username and password.  
For example, to check if the `.hdarc` file exists in the $HOME directory, open a terminal and run the following command:

In [ ]:
ls $HOME/.hdarc

Then, copy the code below into the file `$HOME/.hdarc` (in your Unix/Linux environment) and replace the placeholders with your WEkEO account credentials:

```yaml
user_name: #############
password: #############

## Process data with HDA Client

### Search for the Dataset ID from the WEkEO Landing Platform

<a href='https://wekeo.eu/' target='_blank'>WEkEO</a> offers access to a vast amount of data. Under <a href='https://wekeo.eu/data' target='_blank'>WEkEO DATA</a>, clicking the "+" to add a layer opens a catalog search.  
Here, you can use free text or the filter options on the left to refine your search by satellite platform, sensor, Copernicus service, area (region of interest), general time period (past or future), and various other flags.

<img src='../../media/land/WEkEO_data_01.jpg' align='middle' height='400px'></img>

You can click on the datasets you are interested in to view detailed information, including the dataset's temporal and spatial extent, collection ID, and metadata.

When searching for Sentinel-2 products, click under "Platform" in the Filters on the left-hand side of the catalog panel.  
Two datasets are available, but we will use “SENTINEL-2 Level-1C”. Once you have found it, select 'Details' to read the dataset description. 

The dataset description provides the following information:
* Abstract: A general description of the dataset.
* Classification: Including the Dataset ID.
* Resources: Links to the Product Data Format Specification guide, and JSON metadata.
* Contacts: Information about the data source from its provider.
* Raw Metadata: Details of the dataset in XML format.

<img src='../../media/land/WEkEO_data_02.jpg' align='centre' height='400px'></img>

You will need this information to request data from the Harmonised Data Access API.

This process is explained in a previous training session, which can be found on the <a href='https://www.youtube.com/channel/UCvS3VvKmMKs1M2ZkmQPyRlw' target='_blank'>WEkEO YouTube Channel</a>. The YouTube channel also contains many other useful training and support materials,  
such as how to <a href='https://www.youtube.com/watch?v=pmCkvXcnZxY&list=PLAT-b7DuvMgogqJa5_ii5GteOYmXCce24&index=2' target='_blank'>clone the GitHub repository to refresh the training materials</a>.

For this session, the details of the required datasets have already been prepared as JSON files, which will be used below.

In [ ]:
dataset_id_S2 = "EO:EO:ESA:DAT:SENTINEL-2:MSI"
dataset_id_corine = "EO:CLMS:DAT:CORINE"

filename_json_S2 = os.path.join(os.getcwd(), '../../data/raw/land/S2_request.json')
filename_json_corine = os.path.join(os.getcwd(), '../../data/raw/land/corine_corsica.json')

In order to interact with WEkEO's Harmonised Data Access API, each user first makes sure the file `$HOME/.hdarc` exists with the URL to the API end point and your user and password.  
For example, to search for the file .hdarc in the $HOME diretory, the user would open a terminale and run the following command:

In [ ]:
!find $HOME -type f -name .hdarc

Then he could copy the code below in the file `$HOME/.hdarc` (in your Unix/Linux environment) and adapt the following template with the credentials of your WEkEO account:

```python
url: 'https://wekeo-broker.prod.wekeo2.eu/databroker'
user_name = #############
password = #############

If he doesn't have a WEkEO account, please self register at the <a href='https://my.wekeo.eu/web/guest/user-registration' target='_blank'>WEkEO registration page</a>.

### Load Data Descriptor File and Request Data


The Harmonised Data Access API can read your data request from a JSON file. In this JSON file, you can specify the dataset you want to download.  
The file is essentially a dictionary and can include the following keys:

- **datasetID**: The dataset's collection ID.
- **stringChoiceValues**: The type of dataset, e.g., 'Non Time Critical'.
- **dataRangeSelectValues**: The time period for which you want to retrieve data.
- **boundingBoxValues**: Optional, to define a subset of a global field.

You can also obtain a specific example of a JSON file for a particular query from the WEkEO DATA portal.

### Displaying a JSON Query from a Request Made to the Harmonised Data Access API Through the Data Portal

You can load the JSON file using `json.load()`. Alternatively, you can copy and paste the dictionary describing your data directly into a cell, as demonstrated in the YouTube video.

For this training session, multiple JSON files have already been prepared to select the appropriate Sentinel-2 scene and CLC data for the subsequent tasks. The details were loaded earlier in the notebook. 

The following cell reads these JSON files and displays their contents.  

In [72]:
try:
    with open(filename_json_S2, 'r') as f:
        data_S2 = json.load(f)
        print('Your JSON file:')
        print(json.dumps(data_S2, indent=4))
except:
    print('Your JSON file is not in the correct format, or is not found, please check it!')

Your JSON file:
{
    "datasetId": "EO:ESA:DAT:SENTINEL-2:MSI",
    "boundingBoxValues": [
        {
            "name": "bbox",
            "bbox": [
                9.425764239078317,
                42.74275713340862,
                9.735642520957134,
                43.05969192516483
            ]
        }
    ],
    "dateRangeSelectValues": [
        {
            "name": "position",
            "start": "2017-08-02T00:00:00.000Z",
            "end": "2017-08-02T23:00:00.000Z"
        }
    ],
    "stringChoiceValues": [
        {
            "name": "processingLevel",
            "value": "LEVEL2A"
        }
    ]
}


In [73]:
try:
    with open(filename_json_corine, 'r') as f:
        data_corine = json.load(f)
        print('Your JSON file:')
        print(json.dumps(data_corine, indent=4))
except:
    print('Your JSON file is not in the correct format, or is not found, please check it!')

Your JSON file:
{
    "datasetId": "EO:CLMS:DAT:CORINE",
    "stringChoiceValues": [
        {
            "name": "product_type",
            "value": "Corine Land Cover 2018"
        },
        {
            "name": "format",
            "value": "GeoTiff100mt"
        }
    ]
}


### Download Requested Data

You can use the client directly to download the data, as shown in the following example.

In [ ]:
c = Client()

download_dir_path = os.path.join(os.getcwd(), '../../data/download/land')

matches = c.search(data_S2)
print("Sentinel 2:")
print(matches)
matches.download(download_dir_path)

matches = c.search(data_corine)
print("\nCorine Land Cover:")
print(matches)
matches.download(download_dir_path)

### Decompressing Sentinel-2 and Corine Land Cover Data

In [ ]:
#rename SAFE files

for item in os.listdir(download_dir_path): 
    if item.endswith('SAFE'):
        os.rename(os.path.join(download_dir_path, item), os.path.join(download_dir_path, item +".zip"))

If you encounter permission issues when renaming the `.SAFE` file with the code above, you can manually rename it. Right-click the file, select "Rename," and add `.zip` to the end of the filename.  
For example, rename `S2A_MSIL2A_20170802T101031_N0205_R022_T32TNN_20170802T101051.SAFE` to `S2A_MSIL2A_20170802T101031_N0205_R022_T32TNN_20170802T101051.SAFE.zip`

In [ ]:
processing_dir_path = os.path.join(os.getcwd(), '../../data/processing/land')

extension = ".zip"
for item in os.listdir(download_dir_path): 
    print("Decompressing " + item + " ... ", end = '')
    if item.endswith(extension): 
        file_name = os.path.join(download_dir_path, item) 
        with zipfile.ZipFile(file_name, 'r') as zip_ref:
            zip_ref.extractall(processing_dir_path)
    print("DONE")

## Cleanup

To ensure a clean workspace and remove all downloaded files and processing artifacts created during this session, run the following code. This will delete any files that were downloaded and processed within this notebook.

In [ ]:
import os
import shutil

paths_to_cleanup = [
    download_dir_path,
    processing_dir_path
]

for path in paths_to_cleanup:
    if os.path.isfile(path):
        os.remove(path)
    elif os.path.isdir(path):
        shutil.rmtree(path)

print("Cleanup complete. All downloaded and processed files have been removed.")

<hr>

<p><img src='../../media/land/all_partners_wekeo_2.png' align='left' alt='Logo EU Copernicus' height='400px'></img></p>